In [1]:
import os
import gc
import cv2
import timm
import random
import numpy as np
import pandas as pd
from glob import glob

import torch
import torch.nn as nn
from torch import Tensor
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.autonotebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import wandb
from utils import FocalLoss, LabelSmoothingLoss, F1Loss

import warnings
warnings.filterwarnings(action='ignore') 

/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [3]:
CFG = {
    # 원본 (512, 384)
    'IMG_SIZE_H': 512,
    'IMG_SIZE_W': 384,
    'EPOCHS': 100,
    'LEARNING_RATE': 2e-5,
    'BATCH_SIZE': 32,
    'SEED': 909,
    'PAATIENCE_LIMIT': 5,
    'MODEL': 'efficientnet_b3',
    'LOSS': 'CrossEntropy',
}

In [4]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="mask-classification",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": CFG['LEARNING_RATE'],
    "architecture": CFG['MODEL'],
    "epochs": CFG['EPOCHS'],
    "batch_size" : CFG['BATCH_SIZE'],
    "loss" : CFG['LOSS'],
    },
    
    name=f"{CFG['MODEL']}_{CFG['LOSS']}_{CFG['IMG_SIZE_H']}-{CFG['IMG_SIZE_W']}"
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kgw5430. Use `wandb login --relogin` to force relogin


In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

### Custom Dataset

In [6]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        self.feature = []
        
        for img_path in self.img_path_list:
            image = cv2.imread(img_path)
            if self.transforms is not None:
                image = self.transforms(image=image)['image']
            self.feature.append(image)
        
    def __getitem__(self, index):        
        if self.label_list is not None:
            return self.feature[index], self.label_list[index]
        else:
            return self.feature[index]
        
    def __len__(self):
        return len(self.img_path_list)

In [7]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE_W'],CFG['IMG_SIZE_W']),
                            A.HorizontalFlip(always_apply=False, p=0.5),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE_W'],CFG['IMG_SIZE_W']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

### Model Define

In [8]:
class CustomModel(nn.Module):
    def __init__(self, num_classes=18, pretrained=True):
        super(CustomModel, self).__init__()
        self.num_classes = num_classes
        self.pretrained = pretrained
        
        # self.model = timm.create_model('efficientnet_b0', pretrained=self.pretrained)
        self.model = timm.create_model(CFG['MODEL'], pretrained=self.pretrained)
        self.fc = nn.Sequential(nn.Dropout(p=0.2, inplace=True),
                               nn.Linear(1000, 512),
                               nn.Dropout(p=0.2, inplace=True),
                               nn.Linear(512, num_classes),
                               )

    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        return x

### Train

In [9]:
def train(model, optimizer, train_loader, val_loader, scheduler, device, criterion, model_name):
    model.to(device)

    best_val_loss = np.inf
    best_val_score = 0
    best_model = None
    
    # Early Stop
    patience_limit = CFG['PAATIENCE_LIMIT']
    patience = 0
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
             
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_acc = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val ACC : [{_val_acc:.5f}]')
        wandb.log({f"Epoch": epoch, f"{model_name}/Train_Loss": _train_loss, f"{model_name}/Val_Loss": _val_loss, f"{model_name}/Val_ACC": _val_acc})
        
        if scheduler is not None:
            scheduler.step(_val_loss)
            
        if best_val_loss > _val_loss:
            best_val_loss = _val_loss
            best_val_score = _val_acc
            best_model = model
            patience = 0
        else:
            patience += 1
            if patience >= patience_limit:
                break

    print(f'Best Loss : [{best_val_loss:.5f}] Best ACC : [{best_val_score:.5f}]')
    return best_model, best_val_score, best_val_loss

In [10]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            logit = model(imgs)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
        _val_acc = f1_score(trues, preds, average='weighted')
    
    return _val_loss, _val_acc

### Load Data

In [11]:
train_dir = '/opt/ml/input/data/train'

In [12]:
all_img_path = glob(os.path.join(train_dir, 'images', '*', '*'))

train_df = pd.DataFrame(columns=['id', 'path', 'mask_label', 'gender_label', 'age_label', 'label'])
train_df['path'] = all_img_path

In [13]:
_file_names = {"mask1": 0, "mask2": 0, "mask3": 0, "mask4": 0, "mask5": 0, "incorrect_mask": 1, "normal": 2}
train_df['id'] = train_df['path'].apply(lambda x : (str(x).split('/')[7]).split('_')[0])
train_df['mask_label'] = train_df['path'].apply(lambda x : _file_names[os.path.splitext(x.split('/')[-1])[0]])
print(train_df['mask_label'].value_counts())

0    13500
2     2700
1     2700
Name: mask_label, dtype: int64


In [14]:
_gender_labels = {"male": 0, "female": 1}
train_df['id'] = train_df['path'].apply(lambda x : (str(x).split('/')[7]).split('_')[0])
train_df['gender_label'] = train_df['path'].apply(lambda x : _gender_labels[(str(x).split('/')[7]).split('_')[1]])

train_df['gender_label'].value_counts()

1    11606
0     7294
Name: gender_label, dtype: int64

In [15]:
train_df['id'] = train_df['path'].apply(lambda x : (str(x).split('/')[7]).split('_')[0])
train_df['age_label'] = train_df['path'].apply(lambda x : int((str(x).split('/')[7]).split('_')[3]))

train_df['age_label'].loc[train_df['age_label'] < 30] = 0
train_df['age_label'].loc[(train_df['age_label'] >= 30) & (train_df['age_label'] < 60)] = 1
train_df['age_label'].loc[train_df['age_label'] >= 60] = 2

train_df['age_label'].value_counts()

0    8967
1    8589
2    1344
Name: age_label, dtype: int64

In [16]:
train_df['label'] = train_df.apply(lambda x : (x['mask_label'] * 6 + x['gender_label']*3 + x['age_label']), axis=1)

### Class Weights

In [17]:
from sklearn.utils.class_weight import compute_class_weight

train_labels = [i for i in train_df['label']]
train_labels.sort()
train_labels_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_df['label']), y=train_labels)
train_labels_weights = torch.FloatTensor(train_labels_weights).to(device)

train_labels_weights

tensor([ 0.3825,  0.5122,  2.5301,  0.2869,  0.2570,  1.9266,  1.9126,  2.5610,
        12.6506,  1.4344,  1.2852,  9.6330,  1.9126,  2.5610, 12.6506,  1.4344,
         1.2852,  9.6330], device='cuda:0')

In [18]:
train_, val_, _, _ = train_test_split(train_df, train_df['label'], test_size=0.2, random_state=CFG['SEED'], stratify=train_df['label'])
print(len(train_), len(val_))

15120 3780


### Train

In [19]:
project_idx = len(glob('/opt/ml/models/*'))

In [21]:
model_name = 'ALL'

train_dataset = CustomDataset(train_['path'].values, train_['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=8)

val_dataset = CustomDataset(val_['path'].values, val_['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=8)

model = CustomModel()
model = nn.DataParallel(model)
model.eval()

optimizer = torch.optim.AdamW(params = model.parameters(), lr=CFG['LEARNING_RATE'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs',min_lr=1e-9, verbose=True)
criterion = nn.CrossEntropyLoss(weight=train_labels_weights).to(device)
# criterion = FocalLoss(weight=train_labels_weights).to(device)
mask_model, mask_best_score, mask_best_loss = train(model, optimizer, train_loader, val_loader, scheduler, device, criterion, model_name)

os.makedirs(f'/opt/ml/models/{project_idx}/{model_name}', exist_ok=True)
torch.save(mask_model.module.state_dict(), f'/opt/ml/models/{project_idx}/{model_name}/[{CFG["MODEL"]}]_[score{mask_best_score:.4f}]_[loss{mask_best_loss:.4f}].pt')

gc.collect() # python 자원 관리 
torch.cuda.empty_cache() # gpu 자원관리

### Inference

In [21]:
test_dir = '/opt/ml/input/data/eval'

In [22]:
df = pd.read_csv(test_dir + '/info.csv')

In [23]:
image_paths = [os.path.join(test_dir, 'images', img_id) for img_id in df.ImageID]

test_dataset = CustomDataset(image_paths, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=8)

In [24]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.to(device)
            
            logit = model(imgs)

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [25]:
model_weights = torch.load(glob(f'/opt/ml/models/{project_idx}/ALL/*')[0])
model = CustomModel()
model.load_state_dict(model_weights)

<All keys matched successfully>

In [26]:
model_preds = inference(model, test_loader, device)

OSError: [Errno 12] Cannot allocate memory

In [30]:
df['ans'] = model_preds
df.to_csv(os.path.join(test_dir, 'submits', f'{CFG["MODEL"]}_{project_idx}.csv'), index=False)
print(f'{CFG["MODEL"]}_{project_idx}.csv - test inference is done!')

efficientnet_b0_3.csv - test inference is done!


### Check Wrong Prediction

In [31]:
val_dataset = CustomDataset(val_['path'].values, val_['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=8)

In [32]:
target, prediction = [], []
with torch.no_grad():
    for imgs, labels in tqdm(iter(val_loader)):
        imgs = imgs.float().to(device)
        labels = labels.to(device)
        
        logit = model(imgs)        
        pred = logit.argmax(dim=1)
        
        indices = (pred != labels).nonzero().squeeze()
        if indices.nelement() != 0:
            target.append(imgs[indices])
            prediction.append((pred[indices], labels[indices]))

In [33]:
print(len(val_))
print(len(target))

3780
84
